In [1]:
from shapely.ops import nearest_points
from shapely.geometry import Point
import geopandas as gpd

print(nearest_points(Point(0,2), Point(1,4))[1])  #Crea tupla in cui butta primo e secondo numero

A=gpd.GeoDataFrame()
A["geometry"]=[Point(1,4), Point(0,3)]

##Serve passare per dataframe per fare unary union (non so how altrimenti, sicuro c'è modo)
u=nearest_points(Point(0,2), A.geometry.unary_union)[1]
print(u)

nearest=A["geometry"]==u
print(nearest)

print(A[nearest])  
#print(A[u])

#Metti tutto insieme, e trovi la stazione giusta

#Giusto come nota:  stations[[True]*20+[False]*16]["station"].values[0]

POINT (1 4)
POINT (0 3)
0    False
1     True
Name: geometry, dtype: bool
                  geometry
1  POINT (0.00000 3.00000)


In [2]:
A.geometry

0    POINT (1.00000 4.00000)
1    POINT (0.00000 3.00000)
Name: geometry, dtype: geometry

# Parte machine learning

In [4]:
import numpy as np
import geopandas as gpd
import pandas as pd

# funzioni di sk-learn
from sklearn.pipeline import Pipeline
from sklearn.ensemble import RandomForestClassifier, RandomForestRegressor
from sklearn.neighbors import KNeighborsRegressor
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import GridSearchCV, cross_validate
from sklearn.linear_model import Ridge
from sklearn.preprocessing import OneHotEncoder, StandardScaler, RobustScaler
from sklearn.metrics import matthews_corrcoef, r2_score, accuracy_score, confusion_matrix, plot_confusion_matrix
from sklearn.model_selection import train_test_split

# custom lib
import Supporto_ML as supp_ML



In [7]:
#Works as expected, riesce a fittare la funzione identità tra 0 e 10 con circa 15 dati
#MA SE AUMENTO IL NUMERO MASSIMO aggiungere l'encoder rovina di violenza la stima (prende la colonna int come onehot)
    #QUESTO VA FIXATO
N=500
mx=1000
x=np.floor(mx*np.random.rand(N)).astype(int)

y=x
x=x.reshape(-1, 1)




pipe_logistic = Pipeline([
    ('encoder', OneHotEncoder(sparse=False, handle_unknown='ignore')),
    ('scaler', StandardScaler()),
    ('regressor', LogisticRegression())
])

pipe_logistic = pipe_logistic.fit(x, y)

x_test=np.floor(mx*np.random.rand(N))
x_test=np.array([int(i) for i in x_test])
x_test=x_test.reshape(-1, 1)


y_pred = pipe_logistic.predict(x_test)


print("score=%f" % r2_score(y_pred, x_test))
#np.transpose(x_test)-y_pred

score=-1.012098


In [6]:
#AGGIUNGERE COLONNE DI FODDER PEGGIORA SENSIBILMENTE IL VALORE DELLO SCORE

#In questo esempio la funzione è y=first_line(x), quindi per ogni columns>1 ho una colonna di spazzatura
#Già con N=50 la pipeline non riesce a trovare la relazione
N=1000
mx=1000
columns=30
x=np.floor(mx*np.random.rand(N,columns)).astype(int)
y=[i[0] for i in x]


pipe_logistic = Pipeline([
    ('encoder', OneHotEncoder(sparse=False, handle_unknown='ignore')),
    ('scaler', StandardScaler()),
    ('regressor', LogisticRegression(max_iter=10000))
])

pipe_logistic = pipe_logistic.fit(x, y)

x_test=np.floor(mx*np.random.rand(N,columns)).astype(int)
y_test=[i[0] for i in x_test]

y_pred = pipe_logistic.predict(x_test)


print("score=%f" % r2_score(y_pred, y_test))


score=0.283534
